In [1]:
%%capture
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE db;'

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
user = 'postgres'
password = 'postgres'
host = 'localhost'
post = 5432
name = 'db'

In [4]:
point = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, post, name)

In [5]:
con = create_engine(point)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [7]:
df_test = pd.read_csv('/content/data_two_columns.csv')

In [8]:
df_test.to_sql("sales_test", con, if_exists='replace', index=False, method='multi')

In [9]:
def select_postgresql(sql):
    return pd.read_sql(sql, con)

In [10]:
sql = """select s.* from sales_test as s limit 10"""

In [11]:
print(select_postgresql(sql))

  invoiceno stockcode
0    536365    85123A
1    536365     71053
2    536365    84406B
3    536365    84029G
4    536365    84029E
5    536365     22752
6    536365     21730
7    536366     22633
8    536366     22632
9    536367     84879


In [12]:
%%capture
!pip install pyspark

In [13]:
from pyspark.sql import SparkSession

In [14]:
!wget https://jdbc.postgresql.org/download/postgresql-42.3.3.jar

--2022-03-21 11:54:30--  https://jdbc.postgresql.org/download/postgresql-42.3.3.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1039047 (1015K) [application/java-archive]
Saving to: ‘postgresql-42.3.3.jar’

postgresql-42.3.3.j 100%[===================>]   1015K  --.-KB/s    in 0.1s    

2022-03-21 11:54:31 (6.62 MB/s) - ‘postgresql-42.3.3.jar’ saved [1039047/1039047]



In [15]:
spark = SparkSession \
    .builder \
    .appName("Postgresql") \
    .config("spark.jars", "/content/postgresql-42.3.3.jar") \
    .getOrCreate()

In [ ]:
# df = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql://localhost:5432/db")\
#     .option("dbtable", "sales_test") \
#     .option("user", "postgres") \
#     .option("password", "postgres") \
#     .option("driver", "org.postgresql.Driver") \
#     .load()

In [19]:
df = spark.read.jdbc(url = "jdbc:postgresql://localhost:5432/db", 
                     table = "(SELECT s.* FROM sales_test as s) as tbl",
                     properties={"user": "postgres", "password": "postgres", "driver": 'org.postgresql.Driver'}).createTempView('tbl')

In [24]:
spark.sql("""select t.invoiceno, count(distinct t.stockcode) as count_stockcode
           from tbl as t 
           group by t.invoiceno
           order by count(distinct t.stockcode) desc
           limit 10""").show() 

+---------+---------------+
|invoiceno|count_stockcode|
+---------+---------------+
|   573585|           1110|
|   581219|            749|
|   581492|            731|
|   580729|            721|
|   558475|            704|
|   579777|            687|
|   581217|            676|
|   537434|            674|
|   580730|            661|
|   580367|            650|
+---------+---------------+

